# Permutation tests

In [ ]:
import re
from pathlib import Path

#import ipywidgets as widgets
#from ipywidgets import Layout, Button, HBox, VBox, Box, Dropdown, Select, Text, Output, IntSlider, Label
from IPython.display import display, clear_output, Image, Video
import panel as pn


#from bokeh.plotting import figure, show, output_notebook
#output_notebook()
#pn.extension('bokeh')
pn.extension()
#pn.extension('ipywidgets')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
outputs_root = Path('images_out')
#folder_prefix = 'permutations_limited_palette_2D'
#folder_prefix = 'exp_stability_modes'
folder_prefix = 'exp_video_basic_stability_modes'
folders = list(outputs_root.glob(f'{folder_prefix}_*'))
len(folders)

def format_val(v):
    try:
        v = float(v)
        if int(v) == v:
            v = int(v)
    except:
        pass
    return v

def parse_folder_name(folder):
    #chunks = folder.name[1+len(folder_prefix):].split('_')
    #chunks = folder.name[1+len(folder_prefix):].split('-')
    metadata_string = folder.name[1+len(folder_prefix):]
    pattern = r"_?([a-zA-Z_]+)-(True|False|[0-9.]+)"
    matches = re.findall(pattern, metadata_string)
    d_ = {k:format_val(v) for k,v in matches}
    d_['fpath'] = folder
    d_['n_images'] = len(list(folder.glob('*.png')))
    return d_

#parse_folder_name(folders[0])
df_meta = pd.DataFrame([parse_folder_name(f) for f in folders])

variant_names = [v for v in df_meta.columns.tolist() if v not in ['fpath']]
variant_ranges = {v:df_meta[v].unique() for v in variant_names}
[v.sort() for v in variant_ranges.values()]
True

##########################################

# to do: output and display palettes

#kargs = {k:widgets.Dropdown(options=v, value=v[0], disabled=False, layout=Layout(width='auto')) for k,v in variant_ranges.items()}
#kargs['i'] = widgets.IntSlider(min=1, max=40, step=1, value=1, continuous_update=False, readout=True, readout_format='d')

n_imgs_per_group = 20

def setting_name_shorthand(setting_name):
    return ''.join([tok[0] for tok in setting_name.split('_')])

kargs = {k:pn.widgets.DiscreteSlider(name=k, options=list(v), value=v[0]) for k,v in variant_ranges.items()}
#kargs['i'] = pn.widgets.IntSlider(name='i', start=1, end=n_imgs_per_group, step=1, value=n_imgs_per_group)
kargs['i'] = pn.widgets.Player(interval=300, name='step', start=1, end=n_imgs_per_group, step=1, value=1, loop_policy='reflect')

PRELOAD_IMAGES = False
from PIL import Image

def read_image(fpath):
    #return plt.imread(fpath)
    #return pn.pane.PNG(fpath, width=700)
    with Image.open(fpath) as _img:
       img = _img.copy()
    return img

url_prefix = "https://raw.githubusercontent.com/dmarx/pytti-settings-test/main/images_out/"
#im_path = im_path.replace('images_out/', url_prefix)

image_paths = [str(p) for p in Path('images_out').glob('**/*.png')]
#print(len(list(image_paths)))
d_image_urls = {im_path:im_path.replace('images_out/', url_prefix) for im_path in image_paths}

if PRELOAD_IMAGES:
    d_images = {}
    for folder in df_meta['fpath']:
        for im_path in folder.glob('*.png'):
            d_images[str(im_path)] = read_image(im_path)

In [ ]:
variant_names

In [ ]:


#@widgets.interact(
@pn.interact(
    **kargs
)
#@pn.interact
def display_images(
    palettes,
    palette_size,
    gamma,
    hdr_weight,
    smoothing_weight,
    palette_normalization_weight,
    i,
):
    folder = df_meta[
        (palettes == df_meta['palettes']) &
        (palette_size == df_meta['palette_size']) &
        (gamma == df_meta['gamma']) &
        (hdr_weight == df_meta['hdr_weight']) &
        (smoothing_weight == df_meta['smoothing_weight']) &
        (palette_normalization_weight == df_meta['palette_normalization_weight'])
    ]['fpath'].values[0]
    im_path = str(folder / f"{folder.name}_{i}.png")
    im_url = d_image_urls[im_path]
    #return Image(im_path, width=700)
    #print(type(im_path))
    #im = im_path
    #url_prefix = "https://raw.githubusercontent.com/dmarx/pytti-settings-test/main/images_out/"
    #im_path = im_path.replace('images_out/', url_prefix)
    #print(im_path)
    #if PRELOAD_IMAGES:
    #    im = d_images[im_path]
    #else:
    #    im = im_path
    #return pn.pane.PNG(im, width=700)
    #return im
    #return pn.pane.PNG(im_url, width=700)
    return pn.pane.HTML(f'<img src="{im_url}" width="700">', width=700, height=350, sizing_mode='fixed')

# embedding this makes the page nearly a gigabyte in size.
# need to use a CDN of something like that.
pn.panel(display_images, height=1000).embed(max_opts=n_imgs_per_group, max_states=999999999)
#pn.panel(display_images)
#display_images 

In [ ]:



#@widgets.interact(
@pn.interact(
    **kargs
)
#@pn.interact
def display_images(
    ref,
    dsw,
    ssw,
    i,
):
    folder = df_meta[
        #(reencode_each_frame == df_meta['ref']) &
        #(direct_stabilization_weight == df_meta['dsw']) &
        #(semantic_stabilization_weight == df_meta['ssw'])
        (ref == df_meta['ref']) &
        (dsw == df_meta['dsw']) &
        (ssw == df_meta['ssw'])
    ]['fpath'].values[0]
    im_path = str(folder / f"{folder.name}_{i}.png")
    #im_url = d_image_urls[im_path]
    im_url = im_path
    #return Image(im_path, width=700)
    #print(type(im_path))
    #im = im_path
    #url_prefix = "https://raw.githubusercontent.com/dmarx/pytti-settings-test/main/images_out/"
    #im_path = im_path.replace('images_out/', url_prefix)
    #print(im_path)
    #if PRELOAD_IMAGES:
    #    im = d_images[im_path]
    #else:
    #    im = im_path
    #return pn.pane.PNG(im, width=700)
    #return im
    #return pn.pane.PNG(im_url, width=700)
    return pn.pane.HTML(f'<img src="{im_url}" width="700">', width=700, height=350, sizing_mode='fixed')

# embedding this makes the page nearly a gigabyte in size.
# need to use a CDN of something like that.
pn.panel(display_images, height=1000)#.embed(max_opts=n_imgs_per_group, max_states=999999999)